# Туториал по нейросетевому подходу к численному интегрированию функции одной переменной

__Задача__: использовать нейросеть архитектуры MLP для численного интегрирования функции одной переменной.

__Использованные источники__: 

* Lloyd S., Irani R. A., Ahmadi M. Using neural networks for fast numerical integration and optimization //
IEEE Access. — 2020. — Т. 8. — С. 84519—84531.
* [Документация по Keras](https://www.tensorflow.org/guide/keras)
* [Документация по NumPy](https://numpy.org/doc/)

__Немного теории__

Численным интегралом функции одной переменной называют выражение вида:

$ I(f) = \int_{a}^{b} dx \{f(x)\} $ (1) 

В некоторых случаях неопределенный интеграл функции $f(x)$ может быть задан аналитически,
но иногда аналитическая форма не определяется и тогда прибегают к помощи численных методов.

Применить нейросетевой подход возможно образом, описаным далее.

__1. Нейросеть можно представить аналитически__

Нейросеть, так как она является лишь набором математических операций над числами в определенной последовательности, 
можно представить как некоторую функцию $\hat{f}(x)$, которая может быть задана аналитически:

$ \hat{f}(X) = b^{(2)} + W^{(2)T}\phi(b^{(1)}+W^{(1)}X) $ (2)

-- такое аналитическое представление имеет многослойный перцептрон (MLP) с одним скрытым слоем, где

$ W^{(1)}, W^{(2)} $ - множества весов нейросети,

$ b^{(1)}, b^{(2)} $ - множества смещений нейросети.

$ X $ - множество входных параметров (для функции одной переменной $|X| = 1$ и в дальнейшем будет указываться как $x$),

$ \phi(•) $ - функция активации скрытого слоя.

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
import math

from mpmath import polylog